In [1]:
import torch
from torch.nn import functional as F

In [2]:
import NanoGPTLangugageModel as nano
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [3]:
model = nano.NanoGPTLanguageModel()
model.load_state_dict(torch.load('model_weights.pth'))
m = model.to(nano.device)
encode, decode = nano.encode, nano.decode

In [6]:
start_str = "\n"
idx = torch.tensor(encode(start_str), dtype=torch.long, device=device).unsqueeze(0)
print(decode(m.generate(idx = idx, max_new_tokens=256)[0].tolist()))

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
